# Digital Modulation (ASK-FSK-BPSK)

In [ ]:
from disiple.signals import TimeSignal, Spectrum
import numpy as np
from bokeh.plotting import show
from bokeh.layouts import row, gridplot

In [ ]:
samples_per_bit = 1000
bit_interval = 1e-9 # [seconds per bit]
bit_rate = 1 / bit_interval # [bits per second]
sample_rate = samples_per_bit * bit_rate # [samples per second=Hz]

### Message Signal

In [ ]:
bit_stream = [0, 0, 0, 1, 1, 0, 1, 1]
samples_mod = np.repeat(bit_stream, samples_per_bit)
mod_signal = TimeSignal(samples_mod, sample_rate)
mod_spec = Spectrum.from_timesignal(mod_signal, dB=False)

### Carrier Signal

In [ ]:
amp_carr = 1
freq_carr = 3 * bit_rate
times = np.arange(0, len(bit_stream) * bit_interval, 1 / sample_rate)
samples_carr = amp_carr * np.sin(2*np.pi*freq_carr*times)
carr_signal = TimeSignal(samples_carr, sample_rate)
carr_spec = Spectrum.from_timesignal(carr_signal, dB=False)

### Amplitude Shift Keying

In [ ]:
samples_ask = np.where(samples_mod == 1, samples_carr, 0)
ask_signal = TimeSignal(samples_ask, sample_rate)
ask_spec = Spectrum.from_timesignal(ask_signal, dB=False)

### Frequency Shift Keying

In [ ]:
freq_carr2 = 2 * bit_rate
samples_carr2 = amp_carr * np.sin(2*np.pi*freq_carr2*times)
samples_fsk = np.where(samples_mod == 1, samples_carr, samples_carr2)
fsk_signal = TimeSignal(samples_fsk, sample_rate)
fsk_spec = Spectrum.from_timesignal(fsk_signal, dB=False)

### Binary Phase Shift Keying

In [ ]:
samples_bpsk = np.where(samples_mod == 1, samples_carr, -samples_carr)
bpsk_signal = TimeSignal(samples_bpsk, sample_rate)
bpsk_spec = Spectrum.from_timesignal(bpsk_signal, dB=False)

### Create Figures

In [ ]:
mag_range = (0, amp_carr/2 * 1.05)
freq_range = (0, 2e10)

mod_fig = mod_signal.plot(title='Message Signal', active_inspect=None)
mod_spec_fig = mod_spec.plot(title='Magnitude Spectrum of Message Signal', x_range=freq_range, y_range=mag_range, active_inspect=None)
carr_fig = carr_signal.plot(title='Carrier Signal', active_inspect=None, line_color='olive')
carr_spec_fig = carr_spec.plot(title='Magnitude Spectrum of Carrier Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='olive')
ask_fig = ask_signal.plot(title='ASK Signal', active_inspect=None, line_color='darkgoldenrod')
ask_spec_fig = ask_spec.plot(title='Magnitude Spectrum of ASK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkgoldenrod')
fsk_fig = fsk_signal.plot(title='FSK Signal', active_inspect=None, line_color='crimson')
fsk_spec_fig = fsk_spec.plot(title='Magnitude Spectrum of FSK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='crimson')
bpsk_fig = bpsk_signal.plot(title='BPSK Signal', active_inspect=None, line_color='purple')
bpsk_spec_fig = bpsk_spec.plot(title='Magnitude Spectrum of BPSK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='purple')

### Link Frequency Axes

In [ ]:
from itertools import product
def link_x_axes(figs):
    for fig1, fig2 in product(figs, figs):
        fig1.x_range.js_link('start', fig2.x_range, 'start')
        fig1.x_range.js_link('end', fig2.x_range, 'end')
link_x_axes({mod_spec_fig, carr_spec_fig, ask_spec_fig, fsk_spec_fig, bpsk_spec_fig})

### Display Figures

In [ ]:
plot = row(
    gridplot([mod_fig, carr_fig, ask_fig, fsk_fig, bpsk_fig], ncols=1, width=700),
    gridplot([mod_spec_fig, carr_spec_fig, ask_spec_fig, fsk_spec_fig, bpsk_spec_fig], ncols=1, width=700),
)
show(plot)

In [ ]:
from bokeh.plotting import save
from bokeh.resources import INLINE
save(plot, filename='dm.html', title='Digital Modulation', resources=INLINE)